# Preprocessing the data

"In any Machine Learning process, Data Preprocessing is that step in which the data gets transformed, or Encoded, to bring it to such a state that now the machine can easily parse it. In other words, the features of the data can now be easily interpreted by the algorithm." - https://towardsdatascience.com/data-preprocessing-concepts-fa946d11c825

## Cleaning the Dataset

##### Imports

In [48]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import re # Expresiones regulares
from sklearn import preprocessing # LabelEncoder

###############################StopWords#####################################

from nltk.corpus import stopwords

# Se debe descargar el conjunto de 'Stop Words' la primera vez
import nltk
nltk.download('stopwords')

#Se carga la librería de lematización
import spacy
nlp = spacy.load("en_core_web_sm")

#Se carga la librería de stemming y se inicializa el stemmer
from nltk import SnowballStemmer
snowball = SnowballStemmer('english')

from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

porter = PorterStemmer()
lancaster = LancasterStemmer()

###############################StopWords#####################################

[nltk_data] Downloading package stopwords to C:\Users\Juan
[nltk_data]     José\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data = pd.read_table("../Dataset/labeledEligibilitySample1M.csv", header = None)
uniqueInterventions = 0
uniqueDiagnoses = 0

In [3]:
def takeRandomSamples(n):
    a = data[data[0] == "__label__0"].sample(n)
    b = data[data[0] == "__label__1"].sample(n)
    
    return pd.concat([a,b])

In [4]:
def createDataFrame(table, n, specialCharacters=True):
    df = pd.DataFrame(np.array(table).reshape(n*2,2), columns=['Label', 'Description'])
    
    df['Interventions'], df['Diagnoses'] = df['Description'].str.split('.', 1).str
    df['Eligible'] = df['Label'].str.extract('(\d)', expand=True)

    df = df.drop(['Description'], axis=1)
    df = df.drop(['Label'], axis=1)
    
    df['Interventions'] = df['Interventions'].str.replace("study interventions are\s", "")
    
    # Se transforma todo a minúsculas
    df['Interventions'] = df['Interventions'].str.lower() ##Todo a minúscula
    df['Diagnoses'] = df['Diagnoses'].str.lower()     ##Todo a minúscula

    #Se reemplazan vocales con tildes en ambas variables
    df['Interventions'] = df['Interventions'].str.replace('[áäâà]', 'a', regex=True)
    df['Interventions'] = df['Interventions'].str.replace('[éêèë]', 'a', regex=True)
    df['Interventions'] = df['Interventions'].str.replace('[íïìî]', 'a', regex=True)
    df['Interventions'] = df['Interventions'].str.replace('[óôòö]', 'a', regex=True)
    df['Interventions'] = df['Interventions'].str.replace('[úûùü]', 'a', regex=True)

    df['Diagnoses'] = df['Diagnoses'].str.replace('[áäâà]', 'a', regex=True)
    df['Diagnoses'] = df['Diagnoses'].str.replace('[éêèë]', 'a', regex=True)
    df['Diagnoses'] = df['Diagnoses'].str.replace('[íïìî]', 'a', regex=True)
    df['Diagnoses'] = df['Diagnoses'].str.replace('[óôòö]', 'a', regex=True)
    df['Diagnoses'] = df['Diagnoses'].str.replace('[úûùü]', 'a', regex=True)
    df['Diagnoses'] = df['Diagnoses'].str.replace('^\s', '', regex=True)
    
    if not specialCharacters:
        df['Interventions'] = df['Interventions'].str.replace('[^a-zA-Z# +]', '', regex=True)
        df['Diagnoses'] = df['Diagnoses'].str.replace('[^a-zA-Z# +]', '', regex=True)
    
    return df

In [5]:
def getUniqueInterventions(df):
    return len(np.unique(df['Interventions'])) 

In [6]:
def getUniqueInterventions(df):
    return len(np.unique(df['Diagnoses'])) 

In [7]:
def getInterventionsCount(df):
    return df['Interventions'].value_counts()

In [8]:
def getDiagnosesCount(df):
    return df['Diagnoses'].value_counts()

In [9]:
def transformInterventions(df):
    labels = np.unique(df['Interventions'])

    lb_interventions = preprocessing.LabelEncoder()
    lb_interventions.fit(labels)
    df['Interventions'] = lb_interventions.transform(df['Interventions']) 
    return

# Lemmatization

In [12]:
def lemmatize(df, column):
    samples = df[column]

    samples = [next(nlp(sample).sents).lemma_ for sample in samples]
    
    return samples

# Stemming

https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

In [13]:
def stem(df, column, stemmer='porter'):
    samples = df[column]
    
    if stemmer = 'snowball':
        samples = [snowball.stem(sample) for sample in samples]
    elif stemmer = 'lancaster':
        samples = [lancaster.stem(sample) for sample in samples]
    else: 
        samples = [porter.stem(sample) for sample in samples]
    
    return samples

# Tokenization

In [86]:
def tokenize(df, column):
    return df[column].fillna('').apply(lambda x: x.split())

In [98]:
def detokenize(tokens):
    return ' '.join(tokens)

# Stop Words

In [92]:
def removeStopWords(df, column):
    samples = df[column]
    stop_words = stopwords.words('english')
     
    tokens = tokenize(df, column)
    samples = tokens.apply(lambda x: [item for item in x if item not in stop_words])
    
    for i, sample in enumerate(samples):
        samples[i] = detokenize(sample)
        
    return samples
        
    #return tokens.apply(lambda x: [item for item in x if item not in stop_words])

### Execution

In [100]:
table = takeRandomSamples(5)
df = createDataFrame(table, 5, specialCharacters = False)

df['Interventions'] = lemmatize(df, 'Interventions')
df['Diagnoses'] = lemmatize(df, 'Diagnoses')

df['Interventions'] = stem(df, 'Interventions')
df['Diagnoses'] = stem(df, 'Diagnoses')


df['Interventions'] = removeStopWords(df, 'Interventions')
df['Diagnoses'] = removeStopWords(df, 'Diagnoses')

df

,Interventions,Diagnoses,Eligible
0,afatinib,head neck neoplasm diagnosi,0
1,pv,stage iv squamous cell carcinoma hypopharynx d...,0
2,lenalidomid,follicular lymphoma diagnosis man must also co...,0
3,antibodies monoclon,stage iv pancreatic cancer diagnosis currently...,0
4,docetaxel,sarcoma diagnosis patient must great equal two...,0
5,bevacizumab,renal cancer diagnosis -pron- qualify trial st...,1
6,pioglitazon,advanced melanoma diagnosis know time entry ga...,1
7,pembrolizumab,stage ivb hepatocellular carcinoma diagnosis p...,1
8,cisplatin,metastatic malignant neoplasm lung diagnosis d...,1
9,cyclophosphamid,colorectal cancer diagnosis patient know sensi...,1
